In [1]:
import ast
import numpy as np
from sklearn.preprocessing import StandardScaler
import extract
import SumVector
import AverageVector
import NodesTypes

In [2]:
class Compare(ast.NodeTransformer):
    def __init__(self, juypter_file_1, juypter_file_2, question_index, type_sum=9, path=" "):

        self.type_sum = type_sum
        self.question_index = question_index

        self.nodes_type = NodesTypes.return_nodes_array(type_sum)

        self.code_1 = extract.get_source_without_tests(juypter_file_1, question_index)
        self.code_2 = extract.get_source_without_tests(juypter_file_2, question_index)

        self.node_1 = ast.parse(self.code_1)
        self.node_2 = ast.parse(self.code_2)

        self.vector_1 = self.generic_visit(self.node_1).vector
        self.vector_2 = self.generic_visit(self.node_2).vector



        if type_sum == 35:
            sum_vector = np.array(
                [80641, 21977, 8191, 10157, 1476, 242303, 265217, 60918, 27235,
                 6242, 6242, 53105, 12643, 9745, 17359, 14198, 2935, 2938,
                 69784, 11081, 47510, 17974, 17639, 2325, 1948, 508, 47034,
                 1575, 2297, 5480, 837, 7262, 7262, 15368, 7630])
            self.weight_coefficient = np.log(sum(sum_vector) / sum_vector)
        
        elif type_sum == 9:
            sum_vector = np.array([1.26295e+05, 6.02591e+05, 2.94868e+05, 4.30670e+04, 5.44000e+02, 5.00970e+04,
                                        1.57500e+03, 9.34200e+03, 4.13970e+04])
            self.weight_coefficient = np.log(sum(sum_vector) / sum_vector)

        if path != " ":
            av = AverageVector.CalucalateAverageVector(type_sum, question_index)
            self.average_vector = av.subtract_average(path)

    def generic_visit(self, node):
        """
        :param node:
        :return: node with a new field "vector"
        """
        # add fields to python objects

        node.vector = np.zeros(self.type_sum)
        ast.NodeTransformer.generic_visit(self, node)
        for general_type_index, one_general_type in enumerate(self.nodes_type):
            if (type(one_general_type)).__name__ == "list":
                if type(node) in one_general_type:
                    node.vector[general_type_index] += 1
            else:
                if type(node) == one_general_type:
                    node.vector[general_type_index] += 1
        for a in ast.iter_child_nodes(node):
            # this node will be sum of all children nodes
            # as well as case logic below.
            node.vector += a.vector
        return node

    @staticmethod
    def is_same_num_child(node_1, node_2):
        """Whether it has a common number of child nodes"""
        num_1 = 0
        num_2 = 0

        for ele in ast.iter_child_nodes(node_1):
            num_1 += 1
        for ele in ast.iter_child_nodes(node_2):
            num_2 += 1

        if num_1 == num_2:
            return True
        else:
            return False

    def child_sim_compare(self):
        """compare the simlarity of first child node """
        pass

    @staticmethod
    def standard_scaler(vector):
        scaler = StandardScaler()
        transformed_vector = vector.reshape(-1, 1)
        scaler.fit(transformed_vector)
        vector_after_ss = np.ravel(scaler.transform(transformed_vector))
        # print("{} {}".format("vector after standard_scaler:\n", vector_after_ss))
        return vector_after_ss

    def inverse_document_frequency(self, vector):
        #print(self.sum_vector)
        vector_after_idf = vector * self.weight_coefficient
        # print("{} {}".format("vector after idf:\n", vector_after_idf))
        return vector_after_idf

    def subtract_average(self, vector):
        vector_after_subtract = vector - self.average_vector
        # print("{} {}".format("vector after subtract_average:\n", vector_after_subtract))
        return vector_after_subtract

    def euclidean_distance(self, idf=False, ss=False, subtract_average=False):
        pass

    def cosine_distance(self, idf=False, ss=False, subtract_average=False):
        assert len(self.vector_1) == len(self.vector_2)
        zero_list = np.zeros(len(self.vector_1))
        if (self.vector_1 == zero_list).all() or (self.vector_2 == zero_list).all():
            return float(1) if (self.vector_1 == self.vector_2).all() else float(0)

        x = self.vector_1
        y = self.vector_2

        if idf:
            x = self.inverse_document_frequency(self.vector_1)
            y = self.inverse_document_frequency(self.vector_2)

        if subtract_average:
            x = self.subtract_average(x)
            y = self.subtract_average(y)

        if ss:
            x = self.standard_scaler(x)
            y = self.standard_scaler(y)

        res = np.array(
            [[x[i] * y[i],
              x[i] * x[i],
              y[i] * y[i]]
             for i in range(len(x))])

        cos = sum(res[:, 0]) / (np.sqrt(sum(res[:, 1])) * np.sqrt(sum(res[:, 2])))

        if -1 <= cos <= 0:
            return 0.5 * cos + 0.5
        else:
            return cos

    def edit_tree_distance(self):
        pass

    def visit_name(self, node):
        pass

    def visit_str(self, node):
        pass

In [3]:
path_original = "C:/Users/82569/Desktop/f9a7d1_original.ipynb"
path_1 = "C:/Users/82569/Desktop/f9a7d1_Primary Plagiarism_1.ipynb"
path_2 = "C:/Users/82569/Desktop/f9a7d1_Intermediate plagiarism_1.ipynb"
path_3 = "C:/Users/82569/Desktop/f9a7d1_Advanced plagiarism_1.ipynb"
path_files = "C:/Users/82569/PycharmProjects/dstion/data/Assignment 7/Root/last"

### I artificially created some plagiarized jupyter notebooks, here are the codes:

### original:

In [ ]:
def bisect(f,x0,x1,tolerance):
    
    
    '''
    finds a root of the equation f(x)=0, x0 and x1 are
    the first two values, they have to have different signs!
    It should not matter which one corresponds to a positive 
    value of f. 
    
    The iteration terminates when the length of the interval
    between the upper and lower limit is smaller than the tolerance
    
    The list returned should contain all points the algorithm calculates using the middle of the interval.  
    '''
    m_vals = []
    
    if f(x0) > 0:
        x_plus = x0
        x_minus = x1
    else:
        x_plus = x1
        x_minus = x0
    
    interval = abs(x_plus - x_minus)
        
    while interval >= tolerance:
        m = (x_plus + x_minus) / 2
        f_m = f(m)
        if f(m) < 0:
            x_minus = m
        else:
            x_plus = m
        interval = abs(x_plus - x_minus)
        m_vals.append(m)
    
    return(m_vals)
        

### Primary Plagiarism:

In [17]:
def bisect_pla(f, x_0, x1, tolerance):
    
    # modify fuction name
    
    '''
    finds a root of the equation f(x)=0, x_0 and x1 are
    the first two values, they have to have different signs!
    It should not matter which one corresponds to a positive 
    value of f. 

    The iteration terminates when the length of the interval
    between the upper and lower limit is smaller than the tolerance

    The list returned should contain all points the algorithm calculates using the middle of the interval.  
    '''

    dfc = []
    
    # modify variable name
    
    if f(x_0) > 0:
        # modify variable name
        x_p = x_0
        x_m = x1
    else:
        x_p = x1
        x_m = x_0

    interval = abs(x_p - x_m)
    
    # modify variable name
    
    while interval >= tolerance:
        m = (x_p + x_m) / 2
        f_m = f(m)
        if f(m) < 0:
            x_m = m
        else:
            x_p = m
        interval = abs(x_p - x_m)
        dfc.append(m)

    return (dfc)

### Intermediate plagiarism:

In [ ]:
def bisect_pla(f, x_0, x_1, tolerance):
    '''
    finds a root of the equation f(x)=0, x_0 and x_1 are
    the first two values, they have to have different signs!
    It should not matter which one corresponds to a positive
    value of f.

    The iteration terminates when the length of the interval
    between the upper and lower limit is smaller than the tolerance

    The list returned should contain all points the algorithm calculates using the middle of the interval.
    '''

    
    # Reverse the order of statements in program code
    if f(x_0) <= 0:
        # comment
        x_subtract = x_1
        x_add = x_0
        
    else:
        x_subtract = x_0
        x_add = x_1
        
        
    # Change the variable declaration position in the program    
    m_vals = []
    
    # Add or delete statements in the program
    absolute_value = abs(x_add - x_subtract)
    interval = absolute_value
    
    # comment
    # add comment to prevent looking suspicious
    while interval >= tolerance:
        m = (x_subtract + x_add) / 2
        f_m = f(m)
        if f(m) < 0:
            x_subtract = m
        else:
            x_add = mx_add       
        m_vals.append(m)
        interval = abs(x_add - x_subtract)

    return (m_vals)


### Advanced plagiarism：

In [ ]:
def secant(f,x0,x1,tolerance,returnPoints=False):
    '''
    finds a root of the equation f(x)=0, x0 and x1 are
    the first two values, from them the function calculates
    the next value x2, from x1 and x2 it calculates x3, ...
    
    The termination condition is
    
    abs((x_{i+1}-x{i})/(x_{i+1}+x_{i})) < tolerance
        
    '''
    xa=x0
    xb=x1
    xs=[x0, x1]
    vs=[f(x0), f(x1)]
    while True:
        if abs((xs[-1] - xs[-2])/(xs[-1] + xs[-2])) < tolerance:
            return xs
        df=(vs[-1]-vs[-2])/(xs[-1]-xs[-2])
        
        xnew = xs[-1] - vs[-1]/df
        xs.append(xnew)
        vs.append(f(xnew))

Explanation of terms:
IDF : inverse document frequency. In this paper ,The IDF is calculated as

idf⁡(t,D) = log ( N / |{d∈D:t∈d}| )

with

	N Total number of AST nodes in the corpus  
	|{d∈D:t∈d}| The number of AST nodes corresponding to one specific characteristic words in Corpus.


### Here are the results of the different parameters

### Parameter explanation：

### def __init__(self, juypter_file_1, juypter_file_2, question_index, type_sum=9, path=" "):

### juypter_file_1， juypter_file_2 are two files that compare the similarity

### question_index：the question index in each files that compare the similarity

### type_sum: The dimension of the characteristic vector, this program provides two dimensions, one is 9 dimensions, the other is 35 dimensions

### path： A dictionary of jupyter notebooks of other students in terms of the assignment, this parameter is used in order to calulate the average characteristic vector.

In [ ]:
c_11 = Compare(path_original, path_1, 3, type_sum=9, path=path_files)

In [4]:
print(c_11.vector_1)
print(c_11.vector_2)

#ss represnts standard_scaler

c_11.cosine_distance(idf=False, ss=False, subtract_average=False)

[ 5. 74. 23.  0.  0. 11.  0.  3.  7.]
[ 5. 74. 23.  0.  0. 11.  0.  3.  7.]


1.0

### Parameter explanation：

### cosine_distance(idf=True, ss=False, subtract_average=False)

### idf:  whether do the inverse document frequency weighing.

### ss：whether to standardize the data

### subtract_average: whether subtract the mean from the vector

In [5]:

print(c_11.cosine_distance(idf=True, ss=False, subtract_average=False))
print(c_11.cosine_distance(idf=False, ss=True, subtract_average=False))
print(c_11.cosine_distance(idf=False, ss=False, subtract_average=True))
print(c_11.cosine_distance(idf=True, ss=True, subtract_average=False))
print(c_11.cosine_distance(idf=False, ss=True, subtract_average=True))

1.0000000000000002
0.9999999999999998
1.0
1.0
1.0


In [6]:
c_12 = Compare(path_original, path_1, 3, type_sum=35, path=path_files)
print(c_12.vector_1)
print(c_12.vector_2)
c_12.cosine_distance(idf=False, ss=False, subtract_average=False)

[ 3.  1.  1.  0.  0. 36. 27. 11.  2.  0.  0.  4.  1.  2.  0.  1.  3.  3.
  6.  0.  1.  0.  0.  0.  0.  0. 11.  0.  2.  0.  1.  1.  1.  4.  1.]
[ 3.  1.  1.  0.  0. 36. 27. 11.  2.  0.  0.  4.  1.  2.  0.  1.  3.  3.
  6.  0.  1.  0.  0.  0.  0.  0. 11.  0.  2.  0.  1.  1.  1.  4.  1.]


1.0

In [7]:
print(c_12.cosine_distance(idf=True, ss=False, subtract_average=False))
print(c_12.cosine_distance(idf=False, ss=True, subtract_average=False))
print(c_12.cosine_distance(idf=False, ss=False, subtract_average=True))
print(c_12.cosine_distance(idf=True, ss=True, subtract_average=False))
print(c_12.cosine_distance(idf=False, ss=True, subtract_average=True))

1.0
1.0
1.0
0.9999999999999998
1.0


In [8]:
c_21 = Compare(path_original, path_2, 3, type_sum=9, path=path_files)
print(c_21.vector_1)
print(c_21.vector_2)
c_21.cosine_distance(idf=False, ss=False, subtract_average=False)

[ 5. 74. 23.  0.  0. 11.  0.  3.  7.]
[ 5. 78. 23.  0.  0. 12.  0.  3.  7.]


0.9998669075887722

In [9]:
print(c_21.cosine_distance(idf=True, ss=False, subtract_average=False))
print(c_21.cosine_distance(idf=False, ss=True, subtract_average=False))
print(c_21.cosine_distance(idf=False, ss=False, subtract_average=True))
print(c_21.cosine_distance(idf=True, ss=True, subtract_average=False))
print(c_21.cosine_distance(idf=False, ss=True, subtract_average=True))

0.9994544663635466
0.9998363940150718
0.9826163909953698
0.9989020086636271
0.9899437459081126


In [10]:
c_22 = Compare(path_original, path_2, 3, type_sum=35, path=path_files)
print(c_22.vector_1)
print(c_22.vector_2)
c_22.cosine_distance(idf=False, ss=False, subtract_average=False)

[ 3.  1.  1.  0.  0. 36. 27. 11.  2.  0.  0.  4.  1.  2.  0.  1.  3.  3.
  6.  0.  1.  0.  0.  0.  0.  0. 11.  0.  2.  0.  1.  1.  1.  4.  1.]
[ 3.  1.  1.  0.  0. 38. 28. 12.  2.  0.  0.  4.  1.  2.  0.  1.  3.  3.
  6.  0.  1.  0.  0.  0.  0.  0. 12.  0.  2.  0.  1.  1.  1.  4.  1.]


0.9998373606752234

In [11]:
print(c_22.cosine_distance(idf=True, ss=False, subtract_average=False))
print(c_22.cosine_distance(idf=False, ss=True, subtract_average=False))
print(c_22.cosine_distance(idf=False, ss=False, subtract_average=True))
print(c_22.cosine_distance(idf=True, ss=True, subtract_average=False))
print(c_22.cosine_distance(idf=False, ss=True, subtract_average=True))

0.9995334673910713
0.9998146861644645
0.9868215675617508
0.9993966886944956
0.9900560015891249


In [12]:
c_31 = Compare(path_original, path_3, 3, type_sum=9, path=path_files)
print(c_31.vector_1)
print(c_31.vector_2)
c_31.cosine_distance(idf=False, ss=False, subtract_average=False)

[ 5. 74. 23.  0.  0. 11.  0.  3.  7.]
[ 6. 56. 22.  0.  0.  7.  0.  4.  7.]


0.9958301951505931

In [13]:
print(c_31.cosine_distance(idf=True, ss=False, subtract_average=False))
print(c_31.cosine_distance(idf=False, ss=True, subtract_average=False))
print(c_31.cosine_distance(idf=False, ss=False, subtract_average=True))
print(c_31.cosine_distance(idf=True, ss=True, subtract_average=False))
print(c_31.cosine_distance(idf=False, ss=True, subtract_average=True))

0.9792196026819983
0.995239845844234
0.44628481059928937
0.9551163627463173
0.41517792249228236


In [14]:
c_33 = Compare(path_original, path_3, 3, type_sum=35, path=path_files)
print(c_33.vector_1)
print(c_33.vector_2)
c_33.cosine_distance(idf=False, ss=False, subtract_average=False)

[ 3.  1.  1.  0.  0. 36. 27. 11.  2.  0.  0.  4.  1.  2.  0.  1.  3.  3.
  6.  0.  1.  0.  0.  0.  0.  0. 11.  0.  2.  0.  1.  1.  1.  4.  1.]
[ 4.  1.  1.  0.  0. 27. 22.  7.  2.  0.  0.  3.  1.  1.  0.  1.  4.  4.
  5.  0.  1.  0.  0.  0.  0.  0.  7.  0.  3.  0.  1.  1.  1.  4.  1.]


0.9933505542368752

In [15]:
print(c_33.cosine_distance(idf=True, ss=False, subtract_average=False))
print(c_33.cosine_distance(idf=False, ss=True, subtract_average=False))
print(c_33.cosine_distance(idf=False, ss=False, subtract_average=True))
print(c_33.cosine_distance(idf=True, ss=True, subtract_average=False))
print(c_33.cosine_distance(idf=False, ss=True, subtract_average=True))

0.9686840519625053
0.9924241305685458
0.0013420871388040794
0.9525166901748242
0.49584465795768295
